In [1]:
import json
import os
import re

all_tweets = {}
files = ["tweet_data/tweets_#nfl.txt", "tweet_data/tweets_#gopatriots.txt", "tweet_data/tweets_#gohawks.txt"]

for file in files: 
    f = open(file, "rb") # open a file
    templist = []
    key = file[18:-4]
    for line in f.readlines(): 
        tweet = json.loads(line)
        templist.append(tweet)
    all_tweets.setdefault(key, list)
    all_tweets[key] = templist
    f.close()

In [2]:
from textblob import TextBlob
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
def get_tweet_sentiment(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    return analysis.sentiment.polarity

In [3]:
def feature_extract_perhour(tweetfile, followers_factor=0.001, influence_factor=1.0): 
    # only consider a part of the data, not every tweets in the file
    po_time_min = tweetfile[0]['citation_date']
    po_time_max = tweetfile[-1]['citation_date']
    print po_time_min, po_time_max
    total_hours = int((po_time_max - po_time_min) / 3600) + 1
    tweets_totalnum_perhour = [0 for i in range(total_hours)]
    retweets_num_perhour = [0 for i in range(total_hours)]
    followers_totalnum_perhour = [0 for i in range(total_hours)]
    influence_level_perhour = [0 for i in range(total_hours)]
    sentiment_level_perhour = [0 for i in range(total_hours)]
    
    for tweet in tweetfile: 
        tweet_time = tweet['citation_date']
        hour = int((tweet_time - po_time_min) / 3600)
        tweets_totalnum_perhour[hour] += 1
        retweets_num_perhour[hour] += tweet['tweet']['retweet_count']
        followers_totalnum_perhour[hour] += tweet['author']['followers'] * followers_factor
        if tweet['author'].get('influence_level', None) is None:
            influence_level_perhour[hour] += 0
        else:
            influence_level_perhour[hour] += (tweet['author']['influence_level'] + 0.01) * influence_factor
        if get_tweet_sentiment(tweet['tweet']['text']) > 0:
            sentiment_level_perhour[hour] += 5
        elif get_tweet_sentiment(tweet['tweet']['text']) == 0:
            sentiment_level_perhour[hour] += 1
        else:
            sentiment_level_perhour[hour] -= 5

    X = []
    for i in range(total_hours):
        x = []
        x.append(tweets_totalnum_perhour[i])
        x.append(retweets_num_perhour[i])
        x.append(followers_totalnum_perhour[i])
        x.append(influence_level_perhour[i])
        X.append(x)

    y = sentiment_level_perhour
    return X, y

nfl_X, nfl_y = feature_extract_perhour(all_tweets['#nfl'], followers_factor=0.001, influence_factor=5.0)

1421517546 1423335336


In [4]:
import sklearn.metrics as smt
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(max_depth=4, random_state=0)
regressor.fit(nfl_X, nfl_y)

test_X, test_y = feature_extract_perhour(all_tweets['#gopatriots'], followers_factor=0.001, influence_factor=5.0)
predicted_y = regressor.predict(test_X)
print smt.r2_score(test_y, predicted_y)

1421257541 1423295675
0.774636543371046


In [5]:
import matplotlib.pyplot as plt
import numpy as np
m, b = np.polyfit(test_y, predicted_y, deg=1)
yp = np.polyval([m, b], test_y)
plt.plot(test_y, yp)
plt.grid(True)
plt.scatter(test_y, predicted_y)
plt.xlabel('Ground truth')
plt.ylabel('Prediced values')
plt.title('#gopatriots (ground truth vs. predicted)')
plt.show()

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [6]:
test_X, test_y = feature_extract_perhour(all_tweets['#gohawks'], followers_factor=0.001, influence_factor=5.0)
predicted_y = regressor.predict(test_X)
print smt.r2_score(test_y, predicted_y)
m, b = np.polyfit(test_y, predicted_y, deg=1)
yp = np.polyval([m, b], test_y)
plt.plot(test_y, yp)
plt.grid(True)
plt.scatter(test_y, predicted_y)
plt.xlabel('Ground truth')
plt.ylabel('Prediced values')
plt.title('#gohawks (ground truth vs. predicted)')
plt.show()

1421518778 1423304269
0.8631892782313286
